In [7]:
import pandas as pd

def process_inventory_data(inventory_df, days=7):
    # Convert timestamp to datetime
    inventory_df['timestamp'] = pd.to_datetime(inventory_df['timestamp'])

    # Filter for dates in the past week
    one_week_ago = pd.Timestamp.now() - pd.Timedelta(days=days)
    inventory_df = inventory_df[inventory_df['timestamp'] >= one_week_ago]

    # Sort the filtered data
    inventory_df.sort_values(by=['supplier', 'code', 'timestamp'], inplace=True)

    # Group by supplier and code
    grouped_df = inventory_df.groupby(['supplier', 'code'])

    # Calculate the net delta
    net_delta_df = grouped_df.agg(
        start_stock=('stock_calculation', 'first'),
        end_stock=('stock_calculation', 'last'),
        start_date=('timestamp', 'first'),
        end_date=('timestamp', 'last')
    ).reset_index()

    # Calculate the delta
    net_delta_df['change'] = net_delta_df['end_stock'] - net_delta_df['start_stock']

    # Select relevant columns for the final table
    final_df = net_delta_df[['supplier', 'code', 'start_date', 'end_date', 'change']]

    return final_df
